Import

In [1]:
# Ensure repository root (with 'Source/ai') is on sys.path
from pathlib import Path
import sys

project_root = next((p for p in [Path.cwd(), *Path.cwd().parents] if (p / 'Source' / 'ai').exists()), None)
if project_root and str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

from dotenv import load_dotenv
from langchain_ollama import ChatOllama
from langchain.agents import create_react_agent, AgentExecutor
from langchain import hub
from langchain.tools import Tool
from langgraph.graph import END, StateGraph
import operator
import requests
from Source.ai.Multi_Agent.Source.Main.Tools import poem_tools, sentiment_tools, travel_tools, weather_tools
from Source.ai.Multi_Agent.Source.Main.Agents.Agents_1 import Coordinator_Agent_1, Flight_Agent_1, Hotel_Agent_1, Travel_Agent_1
from Source.ai.Multi_Agent.Source.Main.Agents.Agents_2 import Coordinator_Agent_2, Flight_Agent_2, Hotel_Agent_2, Travel_Agent_2
from Source.ai.Multi_Agent.Source.Main.Agents.Agents_3 import Coordinator_Agent_3, Flight_Agent_3, Hotel_Agent_3, Travel_Agent_3
from typing import TypedDict, Annotated, List, Any, Dict, Literal
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from Source.ai.Multi_Agent.Source.Main.Memory.memory.memory import memory_manager
from Source.ai.Multi_Agent.Source.Main.Memory.memory.long_term_memory import long_term_memory
import chromadb
from chromadb.config import Settings
from sentence_transformers import SentenceTransformer
import uuid
import os

e:\Project_NguyenMinhVu_2211110063\Source\ai\Multi_Agent\Source\Main\Tools\poem_tools.py:8: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = ChatOllama(model="llama3") # <-- Sử dụng model bạn đã kéo về, ví dụ "llama3", "mistral"
c:\Users\minhv\anaconda3\envs\pytorch-env\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Load model

In [ ]:
load_dotenv()

llm = ChatOllama(model="llama3:8b")

class AgentState(TypedDict):
    input: str
    messages: Annotated[List[str], operator.add]
    
prompt = hub.pull("hwchase17/react")

Load tools

In [ ]:
weatherllm_agent = create_react_agent(llm, [weather_tools.weatherllm_tool], prompt) 
weatherllm_agent_executor = AgentExecutor(agent=weatherllm_agent, tools=[weather_tools.weatherllm_tool], verbose=True, handle_parsing_errors=True) 

def call_weatherllm_agent(state: AgentState):
    result = weatherllm_agent_executor.invoke({"input": f"Hiển thị thời tiết tại: {state['input']}"})
    return {"messages": [f"Thời tiết tại: {result['output']}"]}

sentiment_agent = create_react_agent(llm, [sentiment_tools.sentiment_tool], prompt) 
sentiment_agent_executor = AgentExecutor(agent=sentiment_agent, tools=[sentiment_tools.sentiment_tool], verbose=True, handle_parsing_errors=True)

def call_sentiment_agent(state: AgentState):
    result = sentiment_agent_executor.invoke({"input": f"Phân tích cảm xúc của đoạn text sau: {state['input']}"})
    return {"messages": [f"Phân tích cảm xúc: {result['output']}"]} 

poet_agent = create_react_agent(llm, [poem_tools.poem_tool], prompt) 
poet_agent_executor = AgentExecutor(agent=poet_agent, tools=[poem_tools.poem_tool], verbose=True, handle_parsing_errors=True) 
def call_poet_agent(state: AgentState):
    result = poet_agent_executor.invoke({"input": f"Hãy viết một bài thơ về: {state['input']}"})
    return {"messages": [f"Bài thơ: {result['output']}"]}

# weatherapi_agent = create_react_agent(llm, [weather_tools.weatherapi_tool], prompt) 
# weatherapi_agent_executor = AgentExecutor(agent=weatherapi_agent, tools=[weather_tools.weatherapi_tool], verbose=True, handle_parsing_errors=True) 

# def call_weatherapi_agent(state: AgentState):
#     result = weatherapi_agent_executor.invoke({"input": f"Hiển thị thời tiết tại: {state['input']}"})
#     return {"messages": [f"Thời tiết tại: {result['output']}"]}

Set langgraph

In [ ]:
graph_builder = StateGraph(AgentState)

graph_builder.add_node("sentiment_analyzer", call_sentiment_agent)
graph_builder.add_node("poet", call_poet_agent)
graph_builder.add_node("weather_llm", call_weatherllm_agent)
#graph_builder.add_node("weather_api", call_weatherapi_agent)

graph_builder.set_entry_point("sentiment_analyzer")

graph_builder.add_edge("sentiment_analyzer", "poet")
graph_builder.add_edge("poet", "weather_llm")
#graph_builder.add_edge("weather_llm", "weather_api")
#graph_builder.add_edge("weather_api", END)
graph_builder.add_edge("weather_llm", END)


graph = graph_builder.compile()

input_test1 = "Một ngày đẹp trời với bầu trời trong xanh và mặt trời tỏa nắng ấm áp."
input_test2 = "Hãy cho biết thời tiết thành phố Hồ Chí Minh hiện tại."

final_state = graph.invoke({"input": input_test2})
print("Kết quả cuối cùng:")
for message in final_state["messages"]:
    print("- ", message)

MA without memory

In [ ]:
# Create the graph
workflow = StateGraph(AgentState)
workflow.add_node("coordinator", Coordinator_Agent_1.coordinator_agent)
workflow.add_node("travel_agent", Travel_Agent_1.travel_agent)
workflow.add_node("hotel_agent", Hotel_Agent_1.hotel_agent)
workflow.add_node("flight_agent", Flight_Agent_1.flight_agent)

workflow.set_entry_point("coordinator")

def decide_next_agent(state: AgentState):
    if state.get("needs_user_input", False):
        return "END"
    return state.get("current_agent", "coordinator")

workflow.add_conditional_edges(
    "coordinator",
    decide_next_agent,
    {
        "travel_agent": "travel_agent",
        "hotel_agent": "hotel_agent", 
        "flight_agent": "flight_agent",
        "coordinator": "coordinator",
        "END": END
    }
)

workflow.add_edge("travel_agent", "coordinator")
workflow.add_edge("hotel_agent", "coordinator")
workflow.add_edge("flight_agent", "coordinator")

app = workflow.compile()

# HỆ THỐNG CHAT TỰ ĐỘNG - ĐÃ SỬA LỖI
def interactive_chat_system():
    print("🤖 HỆ THỐNG MULTI-AGENT DU LỊCH THÔNG MINH")
    print("=" * 60)
    print("Bạn có thể nhập bất kỳ yêu cầu nào về du lịch!")
    print("• Gõ 'thoát' để kết thúc")
    print("=" * 60)
    
    # Khởi tạo state với messages rỗng
    state = {
        "messages": [],
        "current_agent": "coordinator",
        "needs_user_input": False,
        "conversation_stage": "greeting"
    }
    
    # Chạy lần đầu để có lời chào
    try:
        output = app.invoke(state, config={"recursion_limit": 50})
        print(f"🤖: {output['messages'][-1].content}")
        state = output
    except Exception as e:
        print(f"🤖: Xin chào! Tôi có thể giúp gì cho chuyến đi của bạn?")
        state["messages"] = [AIMessage(content="Xin chào! Tôi có thể giúp gì cho chuyến đi của bạn?")]
        state["needs_user_input"] = True
    
    # Vòng lặp chat chính
    while True:
        try:
            # Nhập input từ user
            user_input = input("\n👤 Bạn: ").strip()
            
            if user_input.lower() in ['exit', 'quit', 'thoát', 'kết thúc']:
                print("🤖: Cảm ơn bạn! Hẹn gặp lại! 👋")
                break
            
            if not user_input:
                print("🤖: Bạn muốn hỏi gì về du lịch ạ?")
                continue
            
            # Thêm user input vào conversation
            new_messages = state['messages'] + [HumanMessage(content=user_input)]
            state['messages'] = new_messages
            state['needs_user_input'] = False
            
            # Xử lý với multi-agent system
            output = app.invoke(state, config={"recursion_limit": 50})
            
            print(f"👤: {user_input}")
            
            # Hiển thị response
            if output['messages']:
                last_message = output['messages'][-1]
                print(f"🤖: {last_message.content}")
            else:
                print("🤖: Tôi có thể giúp gì thêm cho bạn?")
            
            # Cập nhật state
            state = output
            
        except KeyboardInterrupt:
            print("\n🤖: Hẹn gặp lại bạn! 👋")
            break
        except Exception as e:
            print(f"🤖: Có lỗi xảy ra: {e}")
            print("🤖: Hãy thử lại với yêu cầu khác nhé!")
            # Reset state
            state = {
                "messages": [AIMessage(content="Xin lỗi, có lỗi xảy ra. Bạn muốn hỏi gì về du lịch?")],
                "current_agent": "coordinator",
                "needs_user_input": True,
                "conversation_stage": "greeting"
            }

# Chạy hệ thống
if __name__ == "__main__":
    interactive_chat_system()

MA with short term memory

In [ ]:
def create_initial_state():
    return {
        "messages": [],
        "current_agent": "coordinator",
        "needs_user_input": False,
        "conversation_stage": "greeting",
    }

AGENT_MAP = {
    "coordinator": Coordinator_Agent_2.coordinator_agent,
    "travel_agent": Travel_Agent_2.travel_agent,
    "hotel_agent": Hotel_Agent_2.hotel_agent,
    "flight_agent": Flight_Agent_2.flight_agent,
}

def run_multi_agent_chat():
    print("🤖 Multi-Agent with Short-Term Memory (Agents, no Tools)")
    print("=" * 60)
    print("Type 'exit' to quit, 'clear' to reset memory.")

    state = create_initial_state()

    while True:
        if not state["needs_user_input"]:
            step_fn = AGENT_MAP.get(state["current_agent"], Coordinator_Agent_2.coordinator_agent)
            state = step_fn(state)

            last = state["messages"][-1] if state["messages"] else None
            if last and isinstance(last, AIMessage):
                print(f"\n🤖{state['current_agent']}: {last.content}")

            mem = memory_manager.get_memory()
            print(f"   [Memory: {len(mem.conversation_history)} msgs, {len(mem.user_preferences)} prefs]")
            continue

        user_input = input("\n👤 Bạn: ").strip()
        memory_manager.get_memory().add_message("user", user_input)
        
        if user_input.lower() in ["exit", "quit", "thoát"]:
            print("👋 Bye. Memory will persist for this session.")
            break

        if user_input.lower() in ["clear", "xóa", "reset"]:
            memory_manager.get_memory().clear_memory()
            state = create_initial_state()
            print("🧹 Đã xóa memory. Bắt đầu lại.")
            continue

        state["messages"].append(HumanMessage(content=user_input))
        print(f"👤: {user_input}")
        state["needs_user_input"] = False

# To start the chat, run:
run_multi_agent_chat()


Read short term memory

In [ ]:
mem = memory_manager.get_memory()  # hoặc memory_manager.get_memory("user_id")
print(mem.get_context_summary())
print(memory_manager.get_memory())
print(memory_manager.get_memory("user_id"))

for m in mem.conversation_history:
    if m["role"] in ["user", "assistant"]:
        print(f"{m['timestamp']} | {m['role']}: {m['content']}")